# Advanced Information Retrieval - Project

### Organizational infos:
* 30 Points in total
* Deadline: 09.01.2023 23:59

In [141]:
#imports 
import pandas as pd
import numpy as np
import random
import ast

from torch import nn
import torch
from nltk.tokenize import WhitespaceTokenizer


## Movie struct
Create a struct that contains all  the movie relevant Information to work with.
Could be changed at any given time.


**Define all relevante class attributes**

In [183]:
class Movie:
    def __init__(self, id, name, genres):
        self.id = id
        self.name = name
        self.genres = genres
        self.tagsRelevance = [0] * 1128

    def SetTagRelevanceVector(self, tagsRelevance):
        self.tagsRelevance = tagsRelevance
    
    def SetSumRatings(self, userRatings):
        self.rating = userRatings

**Create A List of Movies and a look up Table for the Genres for later usage.**

The Genres could also be used for a similarity measurement so i just saved the just in case.
Also i am not to sure if genre-14 (Imax) should really be inluded. 

In [143]:

def CreateMovieList(lookUpGenre):
    allMovies = {}
    moviedocs = pd.read_csv('movies.csv')
    lookUpIndex = 1

    for i in range(len(moviedocs)):
        movieGenres = moviedocs['genres'][i]
        movieGenresList = movieGenres.split('|')
        movieGenresNumericalList = []
        for word in movieGenresList:
            if word not in lookUpGenre:
                lookUpGenre[word] = lookUpIndex
                lookUpIndex += 1

            movieGenresNumericalList.append(lookUpGenre[word])
        movie = Movie(moviedocs['movieId'][i], moviedocs['title'][i], movieGenresNumericalList)

        allMovies[movie.id] = movie
    return allMovies
    

**Adds the Genom Score relevances to the Movies**

The genom score is the relevances the tags have to the movies. We also have a csv file mapping the tags to an index.
So i created a tags look up table.

In [144]:
def AddGenoScoresToMovies(movieList):
    genomeScores = pd.read_csv('genome-scores.csv')
    movieTagsRelevance = []
    movieId = genomeScores['movieId'][0]
    for i in range(len(genomeScores)):
        if genomeScores['movieId'][i] != movieId:
            movieList[movieId].SetTagRelevanceVector(movieTagsRelevance.copy())
            movieId = genomeScores['movieId'][i]
            movieTagsRelevance.clear()
        
        movieTagsRelevance.append(genomeScores['relevance'][i])
    movieList[movieId].SetTagRelevanceVector(movieTagsRelevance)
    return

def CreateTagsLookUpTable():
    genres = pd.read_csv('genome-tags.csv')
    tagsLookUpTable = {}
    for index in range(len(genres)):
        tagsLookUpTable[genres['tagId'][index]] = genres['tag'][index]
        index += 1
    return tagsLookUpTable

**Using all the Functions above**


well it's not takes a loot of time to add all the relevance scores...

In [145]:
lookUpTableGenres = {}
movies = CreateMovieList(lookUpTableGenres)
AddGenoScoresToMovies(movies)
lookUpTags = CreateTagsLookUpTable()


**Use the tags as a similarity measurement vector.**

Get a sorted List with a given size depending on the count. Uses the Cosine similiarity to calculate the tags similarity between movies.

In [178]:

def calc_cosine(movieOne, movieTwo):

    d_1 = 0
    d_2 = 0
    numerator = 0
    for index in range(len(movieOne)):
        numerator += movieOne[index] * movieTwo[index]
        d_1 += movieOne[index] * movieOne[index]
        d_2 += movieTwo[index] * movieTwo[index]

    denominator = np.sqrt(d_1) * np.sqrt(d_2)
    res = numerator / denominator
    return res


def GetTopTagsCosineSims(count, movies, query):
    
    if query.tagsRelevance.count(0) == 1128:
        return -1
        
    smiliarMovies = {}
    simValues = []
    
    for mID in movies:
        if movies[mID].name == query.name or movies[mID].tagsRelevance.count(0) == 1128:
            continue
        # print(movies[mID].id)
        sim = calc_cosine(movies[mID].tagsRelevance, query.tagsRelevance)
        simValues.append(sim)
        smiliarMovies[sim] = movies[mID].name

    TopSimilar = {}
    for i in range(count):
        v = max(simValues)
        TopSimilar[v] = smiliarMovies[v]
        simValues.remove(v)
    return TopSimilar

def PrintCosineSimilarity(similarMovies):
    if similarMovies != -1:
        print("Query: ", query.name)
        print("similar Movies: ")
        count = 1
        for sim in similarMovies:
            print(count, similarMovies[sim] + ":", sim)
            count += 1
    else:
        print("Query deosn't have tags!")


**Create a query and get/prints the movies with similar tags**

Uses the functions above to print the query result.

In [182]:
query = movies[14]
count = 10

similarMovies = GetTopTagsCosineSims(count, movies, query)
PrintCosineSimilarity(similarMovies)


Query:  Nixon (1995)
similar Movies: 
1 W. (2008): 0.898684222596803
2 Frost/Nixon (2008): 0.8962395213875662
3 All the Way (2016): 0.8767361489669332
4 Game Change (2012): 0.8752736941131876
5 Recount (2008): 0.8728194328037542
6 Jackie (2016): 0.8687068652182196
7 JFK (1991): 0.8653865668089108
8 Selma (2014): 0.8602333401779444
9 Last King of Scotland, The (2006): 0.8600663714796437
10 Primary Colors (1998): 0.8574306675812458


# User Struct ...NOT FINISHED...

A struct that contains the user ratings and also the tags the user choosed for a movie.

In [208]:
class User:
    def __init__(self, id, usersMovieDic):
        self.id = id
        self.usersMovieDic = usersMovieDic

    def SetTag(self, tagsRelevance):
        self.tagsRelevance = tagsRelevance

class UserMovieRating:
    def __init__(self, id, rating):
        self.id = id
        self.rating = rating
        tags = 0

    def SetTags(self, tags):
        self.tags = tags

**Creates a list in User with every movie ranked by the User**

takes 4 minutes...

In [222]:
# userId,movieId,rating,timestamp
def CreateUsers():
    ratings = pd.read_csv('ratings.csv')
    UserDic = {}
    allUserRatings = {}
    userId = ratings['userId'][0]
    for i in range(len(ratings)):
        if ratings['userId'][i] != userId:
            newUser = User(userId, allUserRatings.copy())      
            userId = ratings['userId'][i]
            UserDic[newUser.id] = newUser
            allUserRatings.clear()

        movie = UserMovieRating(ratings['movieId'][i], ratings['rating'][i])
        allUserRatings[movie.id] = movie

    newUser = User(userId, allUserRatings.copy())
    UserDic[newUser.id] = newUser

    return UserDic


users = CreateUsers()


**[not finished part] not every rated movie has a tag vice versa have to rework the User struct**

In [224]:
#userId,movieId,tag,timestamp
def UserDefinedTag(allUsers):
    userTags = pd.read_csv('tags.csv')

    activeUser = allUsers[userTags['userId'][0]]
    tagsToAdd = []  ##first without lookuptable
    
    movieId = userTags['movieId'][0]
    for i in range(len(userTags)):
        if movieId != userTags['movieId'][i] or activeUser.id != userTags['userId'][i]:
            print(i)
            activeUser.usersMovieDic[userTags['movieId'][i]].SetTags(tagsToAdd.copy())
            tagsToAdd.clear()
            movieId = userTags['movieId'][i]
            activeUser = allUsers[userTags['userId'][i]]

        tagsToAdd.append(userTags['tag'][i])
    
    activeUser.userMovieDic[userTags['movieId'][i]].SetTags(tagsToAdd.copy())


UserDefinedTag(users) 


2
4
5
6
7
10
11


KeyError: 164909

# Diffenrent Query ideas
* search depending on tags
* search depending on title similarity with than searches for a similar movie
* seach query for getting ranking based best movie in specific genre